In [2]:
from lxml import etree 
import json
import pandas as pd


tree = etree.parse("price_2024-05-13_11_30_08_379050.xml")

root = tree.getroot()
categories={}

products=pd.DataFrame([], columns=['Code','Category','Subcategory','Name','Price'])

for vendor in root:
    for product in vendor:
        category,subcategory, pname, price, code=None,None,None,None,None
        for e in product:
            if e.tag=='category':
                category=e.text
            if e.tag=='subcategory':
                subcategory=e.text
            if e.tag=='rprice':
                price=float(e.text.replace(',','.'))
            if e.tag=='gname':
                pname=e.text
            if e.tag=='code':
                code=e.text
        if not category in categories:
            categories[category]=[]
        if not subcategory in categories[category]:
            categories[category].append(subcategory)

        if category in ['СПОЖИВЧІ ТОВАРИ І ЕЛЕКТРОНІКА']:
            if subcategory in ['Електросамокати','Клавіатури дротові','Миші дротові','Ноутбуки для бізнесу',
                              'Клавіатури бездротові','Навушники TWS','Смарт годинники',
                              'Комп\'ютери споживчі','Ноутбуки споживчі','Моноблоки','Смартфони',
                              'Планшети','Робочі станції']:
                products.loc[len(products)]=[code,category,subcategory,pname,price]

products.to_csv('erc_products.csv')

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

products.reset_index(drop=True, inplace=True)
products.head(10)



ukrainian_stop_words=open('stopwords_ua.txt','r+').read().split('\n')

tfidf = TfidfVectorizer(stop_words=ukrainian_stop_words)
tfidf_matrix = tfidf.fit_transform(products['Name'])
voc=tfidf.get_feature_names_out()

print(voc[1500:1600])

['soundform' 'space' 'spark' 'spear' 'spectre' 'speed' 'spin' 'sponge'
 'sport' 'sq' 'ssd' 'star' 'starfish' 'starry' 'stealth' 'steelseries'
 'striker' 'strix' 'studio' 'summit' 'sunset' 'surface' 'surge' 'sva'
 'swap' 'swappable' 'swift' 'switch' 't1' 't1000' 't14' 't14s' 't16'
 't3plus' 't40' 't400' 't45hd' 't50' 't60' 't610k2' 't676h' 't676k'
 'taa5508' 'tab' 'tabmax' 'tat1108' 'tat1138' 'tat1207' 'tat1209'
 'tat2206' 'tat3508' 'tcl' 'tech' 'teclast' 'tecno' 'tempest' 'temporal'
 'tft' 'the' 'thin' 'thinkbook' 'thinkpad' 'titan' 'tkl' 'tmp215' 'tmp216'
 'tmp416' 'tmp614' 'tmp614p' 'tmq0108' 'tokyo' 'tone' 'touch' 'toughpad'
 'tour' 'travel' 'travelmate' 'trinity' 'triton' 'true' 'trust' 'ttc'
 'tuf' 'twilight' 'twr' 'tws' 'tws150i' 'tws250i' 'type' 'u5' 'u7' 'u9'
 'ua' 'uhd' 'uk' 'ukr' 'ukrainian' 'ultra' 'um3402ya' 'um5302la']


In [146]:
search_product=tfidf.transform(['Ноутбук 15.6" Intel i5-1235U 8GB F256GB'])
cosine_sim2 = cosine_similarity(search_product, tfidf_matrix)
sim_scores2 = list(enumerate(cosine_sim2[0]))
sim_scores2 = sorted(sim_scores2, key=lambda x: x[1], reverse=True)

for i in sim_scores2[:10]:
    code, category,subcategory,name,price = products.loc[i[0]]
    print(f"{name} {price} P: {i[1]:0.3f}")
    

Ноутбук 2E Imaginary 15.6" FHD IPS AG, Intel i5-1235U, 8GB, F256GB, UMA, Win11, черный 26999.0 P: 0.621
Ноутбук 2E Imaginary 15.6" FHD IPS AG, Intel i5-1235U, 8GB, F256GB, UMA, DOS, черный 19499.0 P: 0.619
Ноутбук 2E Imaginary 15.6" FHD IPS AG, Intel i5-1235U, 8GB, F256GB, UMA, Win11P, черный 27999.0 P: 0.615
Ноутбук HP 250-G9 15.6" FHD AG, Intel i5-1235U, 8GB, F256GB, UMA, Win11, сріблястий 27999.0 P: 0.607
Ноутбук HP 250-G9 15.6" FHD IPS AG, Intel i5-1235U, 8GB, F256GB, UMA, DOS, сріблястий 21999.0 P: 0.594
Ноутбук Lenovo V15-G3 15.6" FHD IPS AG, Intel i5-1235U, 8GB, F256GB, UMA, Win11P, чорний 27599.0 P: 0.573
Ноутбук Lenovo V15-G3 15.6" FHD IPS AG, Intel i5-1235U, 16GB, F256GB, UMA, DOS, чорний 21799.0 P: 0.520
Ноутбук Lenovo V15-G3 15.6" FHD IPS AG, Intel i5-1235U, 16GB, F256GB, UMA, Win11P, чорний 27999.0 P: 0.516
Ноутбук 2E Imaginary 15.6" FHD IPS AG, Intel i5-1235U, 8GB, F512GB, UMA, DOS, черный 19999.0 P: 0.515
Ноутбук HP 15-fd0058ua 15.6" FHD IPS AG, Intel i5-1235U, 16GB, F51

array([0., 0., 0., ..., 0., 0., 0.])